<a href="https://colab.research.google.com/github/MladenGaric/Py/blob/main/Linear_optimization_Investors_count.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color='DarkViolet ' style="font-size:25px"><b>Investment with limitations

We will try to establish an investment scheme for the group of investors. Due to the complex governance, there are some limitations imposed. In column A you will find the name of the asset. In column B (see stocks.csv), you will find the price for the single unit of that asset (price of one stock). Every investor wants to invest in only one asset, if possible, if not, two. The maximum of companies one investor can invest in is, therefore, two. 

Every investor can invest a maximum $23000 and can hold not more than 33 stocks. The goal for this group of investors is to purchase 33 stocks of every asset. Our goal is to determine the minimum number of investors to purchase all 33 stocks from every company listed on the table.

Example: Since the second stock on our list (AZO) costs 2461.63 our first investor can buy a maximum of 9 stocks of that, and she is left with the $4845.33 that she can either buy two stocks of CTXS or 18 stocks of ACN. That way, our first investor maxed out his capital (there are still some coins left in both cases) and didn’t break the limit of owning more than 33 stocks (of any combination of companies).

Find the optimal solution of this problem - **what is the minimum number of investors** who all have $23000, so they can purchase 33 stocks of all 140 assets without breaking the rule of one investor owning more than 33 stocks of any combination of assets.


This problem could be formulated as a binary linear optimization problem. This type of problem is well established from research perspective, and there are algorithms that find a solution in an acceptable time. In Python the optimization part is done with the help of the PuLP library.

In [ ]:
pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 43.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns
import warnings
from pulp import *

In [ ]:
#import stocks.csv

from google.colab import files
uploaded=files.upload()

Saving Stocks.csv to Stocks.csv


In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

MessageError: ignored

In [ ]:
df = pd.read_csv(io.StringIO(uploaded['Stocks.csv'].decode('utf-8')))
df

,Asset Name,Price
0,"NVR, Inc. (XNYS:NVR)",4564.07
1,"AUTOZONE, INC. (XNYS:AZO)",2461.63
2,"CITRIX SYSTEMS, INC. (XMEX:CTXS*)",2103.70
3,PT Aneka Tambang Tbk (XIDX:ANTM),2010.00
4,BOOKING HOLDINGS INC. (XNAS:BKNG),1958.84
...,...,...
135,"UNITED PARCEL SERVICE, INC. (XNYS:UPS)",173.61
136,FEDEX CORPORATION (XNYS:FDX),173.05
137,CME GROUP INC. (XNAS:CME),167.94
138,"KEYSIGHT TECHNOLOGIES, INC. (XNYS:KEYS)",167.66


<font color='DarkViolet ' style="font-size:25px"><b>Mathematical formulation of the problem:

There are n∈ N order proposals, each with $v_i$ stocks (denoted by s) and a total cost of $w_i, 1 \leq i \leq n$. Without restriction, each combination $\left(v_i, w_i\right)$ needs to come from one account, otherwise this order would have to be split into two.

$$
\text { Order }=\left[\begin{array}{lll}
v_1 & \cdots & v_n \\
w_1 & \cdots & w_n
\end{array}\right]
$$



For a given number m∈ N of investors (accounts) we define:


$$
x_{i, j}=\left\{\begin{array}{c}
1, \text { if Order } i \text { is from account } j \\
0, \text { else. }
\end{array}\right.
$$


Let us visualize these variables with a matrix. The columns are investors (accounts), and the rows are the order proposals.

$$
\begin{array}{ccccc}
\backslash & I_1 & I_2 & \cdots & I_m \\
O_1 & x_{1,1} & x_{1,2} & \cdots & x_{1, m} \\
O_2 & x_{2,1} & x_{2,2} & \cdots & x_{2, m} \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
O_n & x_{n, 1} & x_{n, 2} & \cdots & x_{n, m}
\end{array}
$$


A logical split can now be interpreted as the column sum of this matrix.

Splitting of order proposals is allowed if:

Each investor has less than  W=33s:

$$
\sum_{i=1}^n x_{i, j} \cdot v_i \leq W, \quad \forall 1 \leq j \leq m
$$


Each investor spend less has less than  T=$ 23K:

$$
\sum_{i=1}^n x_{i, j} \cdot w_i \leq I, \quad \forall 1 \leq j \leq m
$$

Each order is from exactly one account:

$$
\sum_{j=1}^m x_{i, j}=1, \quad \forall 1 \leq i \leq n
$$

Not every permissible division provides a satisfactory result. If every account is half used, that is permissible, but certainly not optimal. Therefore, **we always want the money from the last account to be as least spend as possibl**e. Remember, our goal is to have as least investors employed as possible.

In [ ]:
total_stocks = range(1, 34)
capital = 23000
max_stocks = 33
assets_count = len(df)
names = df['Asset Name']
prices = df['Price']

prob = LpProblem("Investment Propositions", LpMinimize)

x = LpVariable.dicts("x", (range(assets_count), total_stocks), lowBound = 0, cat=LpInteger)

prob += lpSum([x[i][j] for i in range(assets_count) for j in total_stocks])

# constraints

# columns
for stocks in total_stocks:
  for asset in range(assets_count):
    prob += lpSum(x[asset][stocks] * stocks * prices[asset]) <= capital * x[asset][stocks]

# rows
for asset in range(assets_count):
  prob += lpSum([x[asset][stocks] * stocks for stocks in total_stocks]) == 33 # make sure that we buy ALL 33 stocks from ALL assets

investors = prob.solve()

solution = [
      [
          (asset, stocks, x[asset][stocks].value())
          for stocks in total_stocks
          if x[asset][stocks].value() > 0
      ]
      for asset in range(assets_count)
  ]

total_investors = 0
investors_lst = []

for asset in range(len(solution)):
  orders = solution[asset]
  asset_name = names[asset]
  #print("For asset " + str(asset) + " Name: " + asset_name)
  for asset, stocks, investors in orders:
    #print(str(stocks) + " stock(s) were bought by " + str(investors) + " investor(s)")
    total = stocks * prices[asset]
    #print("Total per investor: " + str(total))
    total_investors += investors
    for investor in range(int(investors)):
      investors_lst.append((asset, stocks, total))

#print(investors_lst)

#print("TOTAL ORDERS: ",str(total_investors)) # This is total num of investors only with stocks from 1 company

# # # COMBINE ORDERS TO MINIMIZE NUMBERS OF INVESTORS #

investors_lst.sort(key=lambda a: a[2], reverse = True)

def check_for_split(a, asset_index, stocks):
  left_capital = capital - a[2]
  left_stocks = max_stocks - a[1]
  if stocks == 0 or a[0] == asset_index: return 0
  if(stocks <= left_stocks and stocks*prices[asset_index] <= left_capital):
    return stocks
  else: return check_for_split(a, asset_index, stocks - 1)


final_orders_per_investor = []
for l in range(len(investors_lst)):
  left_capital = capital - investors_lst[l][2]
  left_stocks = max_stocks - investors_lst[l][1]
  if investors_lst[l][1] == 0: continue
  found = 0
  if(left_stocks > 0 and left_capital > 0):
    for i in reversed(range(len(investors_lst))):
      add_stocks = check_for_split(investors_lst[l], investors_lst[i][0], investors_lst[i][1])
      if(add_stocks > 0):
        found = 1
        final_orders_per_investor.append([investors_lst[l], (investors_lst[i][0], add_stocks, add_stocks * prices[investors_lst[i][0]])])
        investors_lst[i] = (investors_lst[i][0],investors_lst[i][1] - add_stocks, investors_lst[i][2] - add_stocks * prices[investors_lst[i][0]])
        break
  if found == 0: final_orders_per_investor.append([investors_lst[l], (0,0,0)])
    


final_orders_per_investor.sort(key=lambda a: a[0][2] + a[1][2], reverse = True)

for l in range(len(final_orders_per_investor)):
  total_money_spent = final_orders_per_investor[l][0][2] + final_orders_per_investor[l][1][2]
  total_stocks_bought = final_orders_per_investor[l][0][1] + final_orders_per_investor[l][1][1]
  print("Investor " + str(l+1) + " -> Total spent: " + str(total_money_spent) + "; Total stocks: " + str(total_stocks_bought))
  print("ORDER: " + str(final_orders_per_investor[l]))
  

print("TOTAL INVESTORS: ", str(len(final_orders_per_investor)))



/usr/local/lib/python3.8/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Investor 1 -> Total spent: 22987.73; Total stocks: 6
ORDER: [(0, 5, 22820.35), (139, 1, 167.38)]
Investor 2 -> Total spent: 22987.73; Total stocks: 6
ORDER: [(0, 5, 22820.35), (139, 1, 167.38)]
Investor 3 -> Total spent: 22987.73; Total stocks: 6
ORDER: [(0, 5, 22820.35), (139, 1, 167.38)]
Investor 4 -> Total spent: 22987.73; Total stocks: 6
ORDER: [(0, 5, 22820.35), (139, 1, 167.38)]
Investor 5 -> Total spent: 22987.73; Total stocks: 6
ORDER: [(0, 5, 22820.35), (139, 1, 167.38)]
Investor 6 -> Total spent: 22987.73; Total stocks: 6
ORDER: [(0, 5, 22820.35), (139, 1, 167.38)]
Investor 7 -> Total spent: 22984.34; Total stocks: 33
ORDER: [(8, 32, 22816.96), (139, 1, 167.38)]
Investor 8 -> Total spent: 22977.42; Total stocks: 12
ORDER: [(4, 11, 21547.239999999998), (5, 1, 1430.18)]
Investor 9 -> Total spent: 22946.9; Total stocks: 16
ORDER: [(3, 11, 22110.0), (139, 5, 836.9)]
Investor 10 -> Total spent: 22946.9; Total stocks: 16
ORDER: [(3, 11, 22110.0), (139, 5, 836.9)]
Investor 11 -> Tot